## Import Necessary Package

In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import kagglehub

# general setting. do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

## Load Dataset & Prepare Data

In [95]:
# load dataset（from kagglehub）
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

## Fraud/Non-Fraud Transactions

In [96]:
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent:{len(fraud)}, non-fraudulent:{len(nonfraud)}')
print(f'the positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} ({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')

Fraudulent:492, non-fraudulent:284315
the positive class (frauds) percentage: 492/284807 (0.173%)


In [97]:
# Extract features and labels
X = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# Split the dataset into training and testing sets (with stratification)
x_train, x_test, y_train, y_test = train_test_split(
   X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
# 降維
pca = PCA(n_components=25) 
# n_components=10~25此範圍中數值越大 Precision Score越高
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

iso = IsolationForest(contamination=0.0017,n_estimators=300,random_state=RANDOM_SEED)
iso.fit(x_train)
# decision_function 會給每筆資料一個異常分數
anomaly_score_train = iso.decision_function(x_train)
anomaly_score_test = iso.decision_function(x_test)

# === 3. KMeans 聚類標籤 ===
# 只使用正常樣本來訓練群集
normal_train = x_train[y_train == 0]
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
kmeans.fit(normal_train)

# 對整個訓練 & 測試資料預測所屬 cluster
cluster_train = kmeans.predict(x_train)
cluster_test = kmeans.predict(x_test)

# 將 cluster label one-hot encode（也可以直接用 label）
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cluster_train_oh = enc.fit_transform(cluster_train.reshape(-1, 1))
cluster_test_oh = enc.transform(cluster_test.reshape(-1, 1))

# === 4. 合併所有特徵 ===
# 原始特徵 + 異常分數 + KMeans cluster label
x_train_ext = np.hstack((x_train, anomaly_score_train.reshape(-1, 1), cluster_train_oh))
x_test_ext = np.hstack((x_test, anomaly_score_test.reshape(-1, 1), cluster_test_oh))


# XGBoost

In [100]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(n_estimators=175, learning_rate=0.05, max_depth=7, scale_pos_weight=5, random_state=RANDOM_SEED)

xgb_model.fit(x_train_ext, y_train)

y_pred = xgb_model.predict(x_test_ext)

## Result of 混和

In [101]:
def evaluation(y_true, y_pred, model_name="Model"):
   accuracy = accuracy_score(y_true, y_pred)
   precision = precision_score(y_true, y_pred, zero_division=0)
   recall = recall_score(y_true, y_pred)
   f1 = f1_score(y_true, y_pred)

   print(f'\n{model_name} Evaluation:')
   print('===' * 15)
   print('         Accuracy:', accuracy)
   print('  Precision Score:', precision)
   print('     Recall Score:', recall)
   print('         F1 Score:', f1)
   print("\nClassification Report:")
   print(classification_report(y_true, y_pred))

evaluation(y_test, y_pred, model_name="Hybrid")


Hybrid Evaluation:
         Accuracy: 0.9995552590615966
  Precision Score: 0.9661016949152542
     Recall Score: 0.7702702702702703
         F1 Score: 0.8571428571428571

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.97      0.77      0.86       148

    accuracy                           1.00     85443
   macro avg       0.98      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443

